In [1]:
from pygraphblas import *
from itertools import repeat
import random

In [2]:
M = Matrix.from_lists(
    [0, 0, 0, 0,
    1, 1, 1, 1,
    2, 2, 2, 2,
    3, 3, 3, 3,
    4, 4, 4, 4,
    5, 5, 5, 5, 5,
    6, 6, 6,
    7, 7, 7, 7],
    [0, 2, 3, 6,
    1, 2, 3, 7,
    0, 2, 4, 6,
    0, 1, 3, 5,
    0, 2, 4, 6,
    1, 3, 5, 6, 7,
    0, 4, 6,
    1, 3, 5, 7],
    list(repeat(1.0, 32)))

@unary_op(FP64)  
def random_scaler(x):  
    return random.random() * x + 0.0001

In [3]:
def louvain_cluster(graph, max_iters=20):
    if not graph.square:
        raise lib.DimensionMismatch('First input matrix must be square.')
    rows = graph.nrows
    ApAT = graph.transpose()
    k = graph.reduce_vector()
    m = k.reduce_int() / 2.0    
    S = Matrix.identity(BOOL, rows)
    S_row = Vector.from_type(BOOL, rows)
    mask = Vector.dense(BOOL, rows)
    
    vertices_changed = True
    iters = 0
    while vertices_changed and iters < max_iters:
        vertices_changed = False
        for i in range(rows):
            if i in k:
                S.extract_vector(i, out=S_row, desc=descriptor.tooo)
                Mask = Matrix.from_type(BOOL, rows, rows)
                Mask[i,:] = mask
                S.apply(identity, out=S, mask=Mask, desc=descriptor.oocr)
                v = ApAT[:,i]
                w = v.dup()
                w.assign_scalar(-k[i]/m, accum=times)
                v += w
                q = v @ S
                kappa = q.reduce_float(max_monoid)
                t = q[(q == kappa).nonzero()]
                while len(t) != 1:
                    p = t.apply(random_scaler)
                    max_p = p.reduce_float(max_monoid)
                    t = q[(p == max_p).nonzero()]
                S[i,:] = t
                if not t.iseq(S_row):
                    vertices_changed = True
        iters += 1
    return S

def get_louvain_cluster_assignments(cluster_matrix):
    clusters = Vector.from_type(UINT64, cluster_matrix.nrows)
    index_of_vec = Vector.from_type(UINT64, cluster_matrix.ncols)
    for i in range(cluster_matrix.nrows):
        index_of_vec[i] = i
    cluster_matrix.mxv(index_of_vec, out=clusters, semiring=max_second_uint64)
    return clusters

In [4]:
ans = louvain_cluster(M, 10)
get_louvain_cluster_assignments(ans).to_lists()

[[0, 1, 2, 3, 4, 5, 6, 7], [6, 3, 6, 3, 6, 3, 6, 3]]

In [5]:
with open('simulated_blockmodel_graph_50_nodes.tsv') as challenge:
    N = Matrix.from_tsv(challenge, FP64, 50, 50)

In [8]:
from pygraphblas.demo.gviz import draw
labels = get_louvain_cluster_assignments(louvain_cluster(N, 10))
print(labels.to_lists())

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49], [41, 31, 31, 41, 46, 31, 41, 31, 46, 31, 41, 31, 31, 31, 31, 31, 41, 31, 46, 41, 31, 41, 46, 31, 46, 31, 41, 31, 46, 46, 31, 31, 31, 46, 41, 41, 31, 46, 46, 41, 41, 41, 46, 46, 41, 46, 46, 46, 46, 31]]
